In [1]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import numpy as np
from datetime import datetime, timedelta

### Etape 1:
1/ Importation des données.
2/ "URL", "RESPONSE", "HTML" permettent de récupérer le texte du site.
3/ "SOUP" permet d'utiliser les données pour les scraper.

In [2]:
url = "https://www.vlr.gg/matches/results"
response = requests.get(url)
html = response.text
soup = BeautifulSoup(html, 'html.parser')
data = soup.find_all('div')

### Etape 2:
1/ Création du dataframe "df1" recueillant toutes les données.
1.1/ "a" est le dataframe pour la phase de la compétition.
1.2/ "b" est le dataframe pour la compétition.
2/ Les deux dataframes "a" et "b" sont fusionnés dans le df1.

In [3]:
df1 = pd.DataFrame()

a = pd.DataFrame(columns=["Compétition"])
b = pd.DataFrame(columns=["Phase"])

col = soup.find_all('div', class_='match-item-event-series text-of')
for cols in col:
    texts = cols.find_all(string=True, recursive=False)
    text1 = texts[0].strip()
    a = a._append({'Compétition':text1}, ignore_index=True)
col1 = soup.find_all('div', class_='match-item-event text-of')
for cols in col1:
    texts1 = cols.find_all(string=True, recursive=False)
    text2 = texts1[1].strip()
    b = b._append({'Phase':text2}, ignore_index=True)

df1 = b.join(a, how='inner')

In [4]:
df2 = pd.DataFrame()
c = pd.DataFrame(columns=["Team 2"])
d = pd.DataFrame(columns=["Team 1"])
col = soup.find_all('div', class_='match-item-vs-team-name')
for cols in col:
    texts = cols.find_all(string=True, recursive=True)
    text = texts[2].strip()
    text1 = texts[3].strip()
    d = d._append({'Team 1':text}, ignore_index=True)
    c = c._append({'Team 2':text1}, ignore_index=True)

c = c.replace('', np.nan, regex=True)
c = c.dropna()
c = c.reset_index(drop=True)
d = d.replace('', np.nan, regex=True)
d = d.dropna()
d = d.reset_index(drop=True)

df2 = d.join(c, how='inner')
df1 = df1.join(df2, how='inner')

In [5]:
col = soup.find_all('div', class_='match-item-vs-team-score js-spoiler')
df3 = pd.DataFrame()
e = pd.DataFrame(columns=["Score Team 1"])
f = pd.DataFrame(columns=["Score Team 2"])
    
for i, cols in enumerate(col):
    texts = cols.find_all(string=True, recursive=False)
    team = texts[0].strip()
    
    # Ajouter l'élément à 'c'
    if i % 2 == 0:
        e = e._append({'Score Team 1': team}, ignore_index=True)

for i, cols in enumerate(col):
    texts = cols.find_all(string=True, recursive=False)
    team = texts[0].strip()
    
    # Ajouter l'élément à 'c_impairs'
    if i % 2 != 0:
        f = f._append({'Score Team 2': team}, ignore_index=True)

df3 = e.join(f, how='inner')
df1 = df1.join(df3, how='inner')

In [6]:
g = pd.DataFrame(columns=["Date"])

col = soup.find_all('div', class_='ml-eta mod-completed')
for cols in col:
    texts = cols.find_all(string=True, recursive=False)
    text1 = texts[0].strip()
    if 'd' in text1:
        jour, heures = map(int, text1.replace('d','').replace('h', '').replace('m', '').split())
        duree = timedelta(days=jour, hours=heures, minutes=minutes)
        time = datetime.today() - duree
        g = g._append({'Date':time}, ignore_index=True)
    else:
        heures, minutes = map(int, text1.replace('d','').replace('h', '').replace('m', '').split())
        duree = timedelta(hours=heures, minutes=minutes)
        time = (datetime.today() - duree) + timedelta(seconds=15)
        g = g._append({'Date':time}, ignore_index=True)

df1 = df1.join(g, how='inner')
df1

C:\Users\final\AppData\Local\Temp\ipykernel_22816\1270729377.py:16: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  g = g._append({'Date':time}, ignore_index=True)


,Phase,Compétition,Team 1,Team 2,Score Team 1,Score Team 2,Date
0,China Evolution Series Act 2: Selection,Group Stage–Decider (B),FunPlus Phoenix,Rare Atom,2,0,2023-10-24 13:14:23.485666
1,China Evolution Series Act 2: Selection,Group Stage–Decider (A),Bilibili Gaming,Trace Esports,0,2,2023-10-24 10:59:23.487664
2,Sentinels Off-Season $5k Bounty Showmatch,Showmatch–Showmatch,Sentinels,Moist x Shopify,1,2,2023-10-24 02:29:23.488658
3,Crossfire Cup 2023,Contenders #2 Playoffs–Quarterfinals,Movistar Riders,DSYRE,2,0,2023-10-23 19:59:23.489655
4,Superdome 2023: Egypt,GCC And Iraq Online Qualifier–Round 1,Triple Esports Club,Twisted Minds Esports,2,0,2023-10-23 19:59:23.490653
5,Crossfire Cup 2023,Contenders #2 Playoffs–Quarterfinals,Team Aku Aku,UCAM Tokiers,2,1,2023-10-23 16:59:08.490653
6,Superdome 2023: Egypt,GCC And Iraq Online Qualifier–Round 1,The Ultimates,Team Falcons,0,2,2023-10-23 16:59:08.491650
7,China Evolution Series Act 2: Selection,Group Stage–Elimination (B),TYLOO,FunPlus Phoenix,0,2,2023-10-23 12:59:08.492647
8,China Evolution Series Act 2: Selection,Group Stage–Elimination (A),Royal Never Give Up,Trace Esports,2,0,2023-10-23 10:59:08.493645
9,Knights Monthly Gauntlet 2023: October,Playoffs–Grand Final,Argon Black,Turtle Troop,1,2,2023-10-23 00:59:08.494705


In [7]:
df1.Date + timedelta(seconds=15)

0    2023-10-24 13:14:38.485666
1    2023-10-24 10:59:38.487664
2    2023-10-24 02:29:38.488658
3    2023-10-23 19:59:38.489655
4    2023-10-23 19:59:38.490653
5    2023-10-23 16:59:23.490653
6    2023-10-23 16:59:23.491650
7    2023-10-23 12:59:23.492647
8    2023-10-23 10:59:23.493645
9    2023-10-23 00:59:23.494705
10   2023-10-22 21:59:23.495640
11   2023-10-22 21:59:23.496637
12   2023-10-22 19:59:23.496637
13   2023-10-22 19:59:23.497635
14   2023-10-22 16:59:23.498632
15   2023-10-22 13:59:23.499629
16   2023-10-22 10:59:23.500627
17   2023-10-22 08:59:23.500627
18   2023-10-22 06:59:23.501625
19   2023-10-22 06:59:23.502622
20   2023-10-22 03:59:23.503813
21   2023-10-22 02:59:23.504853
22   2023-10-22 02:59:23.504853
23   2023-10-22 00:59:23.505779
24   2023-10-21 23:59:23.506777
25   2023-10-21 23:59:23.507774
26   2023-10-21 23:59:23.507774
27   2023-10-21 21:59:23.508770
28   2023-10-21 21:59:23.509769
29   2023-10-21 19:59:23.510766
30   2023-10-21 18:59:23.511764
31   202